In [1]:
import os

In [2]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\diamond_price_predication_2\\research'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\diamond_price_predication_2'

In [31]:
from dataclasses import dataclass
from pathlib import Path

In [32]:
@dataclass(frozen= True)
class DataTrainingConfig: 
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str
    
    

In [33]:
from diamond_price_predication.constant import *
from diamond_price_predication.utils.common import read_yaml, create_dicectories

In [34]:
class ConfigurationManger: 
    def __init__(self, config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path= SCHEMA_FILE_PATH): 
        
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        
        create_dicectories([self.config.artifacts_root])
        
    def get_data_training_Config(self)-> DataTrainingConfig: 
        
        config=self.config.data_trainer
        schema=self.schema.TARGET_COLUMN
        params=self.params.ElasticNet
        
        create_dicectories([config.root_dir])
        
        
        datatraing_Config=DataTrainingConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            l1_ratio=params.alpha,
            alpha=params.l1_ratio,
            target_column=schema.name
            )
        return datatraing_Config

In [35]:
import pandas as pd
from diamond_price_predication import logger
from sklearn.linear_model import ElasticNet
import joblib

In [38]:
class DataTrainer: 
    def __init__(self, config=DataTrainingConfig): 
        self.config=config
        
    def training(self): 
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)
        
        x_train=train_data.drop([self.config.target_column], axis=1)
        logger.info(f"x_train data : {x_train.head()}\n\n")
        
        x_test=test_data.drop([self.config.target_column], axis=1)
        logger.info(f"x_test data {x_test.head()}\n\n")
        
        y_train=train_data[[self.config.target_column]]
        logger.info(f"y_train data {y_train.head()}\n\n")
        
        y_test=test_data[[self.config.target_column]]
        logger.info(f"y_test data : {y_test.head()}\n\n")
        
        lr=ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio)
        lr.fit(x_train, y_train)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        
        
        
        
    

In [39]:
try: 
    obj=ConfigurationManger()
    trainingConfig=obj.get_data_training_Config()
    modeltraining= DataTrainer(trainingConfig)
    modeltraining.training()
except Exception as e: 
    raise e


[2023-12-13 08:05:18,932 : INFO: common : Yaml file read config\config.yaml]
[2023-12-13 08:05:18,940 : INFO: common : Yaml file read params.yaml]
[2023-12-13 08:05:18,953 : INFO: common : Yaml file read schema.yaml]
[2023-12-13 08:05:18,961 : INFO: common : Directories file is creted artifacts]
[2023-12-13 08:05:18,966 : INFO: common : Directories file is creted artifacts/data_trainer]


[2023-12-13 08:05:19,667 : INFO: 241704367 : x_train data :    carat  cut  color  clarity  depth  table     x     y     z
0   1.51    5      2        3   61.2   58.0  7.38  7.41  4.52
1   0.72    4      3        4   60.5   58.0  5.76  5.84  3.51
2   0.32    5      2        7   61.8   55.0  4.38  4.40  2.72
3   0.23    3      3        6   63.4   58.0  3.89  3.91  2.45
4   0.42    5      1        4   62.2   55.0  4.82  4.86  3.01

]
[2023-12-13 08:05:19,686 : INFO: 241704367 : x_test data    carat  cut  color  clarity  depth  table     x     y     z
0   0.33    5      5        5   62.2   55.0  4.43  4.46  2.77
1   1.04    3      4        5   63.2   56.0  6.47  6.38  4.07
2   1.39    5      6        5   60.7   56.0  7.25  7.30  4.42
3   0.31    5      3        8   61.9   55.0  4.36  4.39  2.71
4   0.73    5      2        4   61.1   57.0  5.79  5.84  3.55

]
[2023-12-13 08:05:19,697 : INFO: 241704367 : y_train data    price
0  11226
1   2979
2    866
3    482
4   1031

]
[2023-12-13 08:05: